# First, extract the frames from the video
Frame extraction code is based on StackOverflow:
https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames

In [7]:
import cv2
import os

names = ["victor", "rik", "tariq"]

for name in names:
    base_folder = "shoe_pics/"
    extractedFolder = base_folder + "extracted/" + name + "/"
    croppedFolder = base_folder + "cropped/" + name + "/"

    if not os.path.exists(extractedFolder):
        os.makedirs(extractedFolder)

    if not os.path.exists(croppedFolder):
        os.makedirs(croppedFolder)


    vidcap = cv2.VideoCapture(name + '.mp4')
    cont, image = vidcap.read()
    count = 0
    while cont:
        cv2.imwrite(extractedFolder + "frame%d.jpg" % count, image)     # save frame as JPEG file      
        cont, image = vidcap.read()
        count += 1
    print("Finished extracting frames of " + name + "'s shoes!")

Finished extracting frames of victor's shoes!
Finished extracting frames of rik's shoes!
Finished extracting frames of tariq's shoes!


# Setup model

In [8]:
# %tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

2022-11-01 17:00:18.166336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 17:00:19.793545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rik/.local/lib/python3.8/site-packages/cv2/../../lib64::/usr/local/lib
2022-11-01 17:00:19.793569: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-01 17:00:19.852719: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one h

2.10.0


In [9]:
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

# Importing libraries

In [10]:
import os
import cv2
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import sys
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Helper functions

In [11]:
def findRange(box):
    # Coords in {TopDist, LeftDist, BotDist, RightDist}
    # Returns a value between -1 and 1 for left and right
    leftDist = box['box'][1]
    rightDist = box['box'][3]
    avg = (rightDist - leftDist) / 2 + (leftDist)
    return avg * 2 - 1
    
def reduceBoxes(boxes, scores, classes):
    '''Function to reduce all the boxes to only the ones being displayed.
    Slightly modified version of Shreyas Vedpathak's version from stackoverflow:
    https://stackoverflow.com/a/68376832/19442761'''
    
    max_boxes_to_draw = boxes.shape[0]
    min_score_thresh=.5
    
    # Loop over all objects
    coordinates = []
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[0][i] > min_score_thresh:
            class_id = int(classes[0][i] + 1)
            coordinates.append({
                "box": boxes[i],
                "score": scores[0][i]
            })

    return coordinates

def cropImg(image, box):
    # box in {TopDist, LeftDist, BotDist, RightDist}, in ratios
    # left, upper, right, and lower
    PIL_image = Image.fromarray(np.uint8(image)).convert('RGB')
    siz = PIL_image.size
    hwhw = np.tile(siz, 2).flatten()

    # Cropping as left, upper, right, and lower
    newbox = np.array([box[1], box[0], box[3], box[2]])
    imgcoords = hwhw * newbox
 
    return PIL_image.crop(imgcoords)

# Setting up and loading the model

In [12]:
%matplotlib inline

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = 'weights/frozen_inference_graph.pb'

# Path to label map file
PATH_TO_LABELS = 'weights/label_map.pbtxt'

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


2022-11-01 17:00:27.973309: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-01 17:00:27.973360: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rik-TM1703): /proc/driver/nvidia/version does not exist
2022-11-01 17:00:27.974264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Running the actual classification and extracting the shoe pictures

In [13]:
# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
for name in names:
    # Define folders to read from and write to
    PATH_TO_IMAGE_FOLDER = base_folder + "extracted/" + name + "/"
    croppedFolder = base_folder + "cropped/" + name + "/"
    
    # Keep track of the amount of extracted shoes
    counter = 0
    
    for filename in os.listdir(PATH_TO_IMAGE_FOLDER):
        if filename.endswith(".jpg"):
            image_read = cv2.imread(os.path.join(PATH_TO_IMAGE_FOLDER, filename))
            image = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)
            image_expanded = np.expand_dims(image, axis=0)

            # Perform the actual detection by running the model with the image as input
            (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],feed_dict={image_tensor: image_expanded})


            coords = reduceBoxes(boxes[0], scores, classes)
            ranges = [findRange(coords[i]) for i in range(len(coords))]
            print(sum(ranges) / len(ranges)) if len(ranges) else print("None found")        
            # Draw the results of the detection (aka 'visulaize the results')
            vis_util.visualize_boxes_and_labels_on_image_array(
                image,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=5,
                min_score_thresh=0.50)

            # All the results have been drawn on image. Now display the image.
            image_og = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)

            for box in coords:
                cropped = cropImg(image_og, box['box'])
                cropped.save(croppedFolder + "cropped%d.jpg" % counter)
                counter += 1

2022-11-01 17:00:28.082272: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-01 17:00:29.078557: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24883200 exceeds 10% of free system memory.
2022-11-01 17:00:29.084553: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24883200 exceeds 10% of free system memory.
2022-11-01 17:00:29.273656: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24883200 exceeds 10% of free system memory.
2022-11-01 17:00:29.278370: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24883200 exceeds 10% of free system memory.


-0.13350991904735565
0.011214375495910645


2022-11-01 17:00:29.389262: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 24883200 exceeds 10% of free system memory.


-0.08316458761692047
0.006154149770736694
-0.2876562476158142
-0.021099984645843506
None found
0.1400371789932251
0.054271578788757324
-0.031650006771087646
0.33864784240722656
-0.04517360031604767
0.08665430545806885
0.11405491828918457
0.08855898678302765
-0.11229652166366577
0.17562580108642578
-0.05099807679653168
0.003516674041748047
0.21931803226470947
-0.10174649953842163
-0.17360135912895203
0.4140193462371826
0.13239893317222595
0.06745903193950653
None found
-0.10811081528663635
-0.004277825355529785
-0.1168266087770462
0.14003720879554749
0.2446954846382141
-0.13691315054893494
-0.17633140087127686
0.024616718292236328
0.1800743043422699
-0.0982297956943512
-0.20238347351551056
-0.44151225686073303
-0.0017583668231964111
None found
0.17549479007720947
-0.08746293187141418
0.0017583370208740234
-0.07330852746963501
0.09110283851623535
0.2281096875667572
-0.03141319751739502
0.21301119029521942
-0.2915901839733124
None found
-0.03328984975814819
-0.10314559936523438
0.25788289

0.0
-0.11229652166366577
-0.19645965099334717
-0.15425963699817657
0.045479923486709595
0.29117274284362793
None found
0.3008437305688858
0.17117726802825928
-0.07009649276733398
-0.3265852630138397
-0.06300678849220276
-0.34448572993278503
-0.007033318281173706
0.08064651489257812
-0.12108814716339111
-0.10350480675697327
-0.10877984762191772
None found
0.24569296836853027
-0.11211982369422913
-0.09999996423721313
-0.08416318893432617
-0.09647145867347717
-0.25600624084472656
0.08152565360069275
0.15338289737701416
-0.03868332505226135
-0.268314465880394
-0.08431793749332428
-0.03241074085235596
0.08943815529346466
0.09043572843074799
0.12272806465625763
0.03771999478340149
-0.14649516344070435
0.3981943130493164
0.14746323227882385
-0.1077822595834732
0.2070096731185913
-0.06854379177093506
0.1087799072265625
0.07009652256965637
-0.031650006771087646
None found
-0.002131342887878418
-0.08382478356361389
0.20173464715480804
0.11334574222564697
-0.04337882995605469
-0.10877984762191772

-0.3333643227815628
-0.20052456855773926
0.16972851753234863
None found
None found
0.006579995155334473
None found
-0.21580135822296143
0.2750455141067505
-0.09307324886322021
-0.20065978169441223
-0.38421061635017395
-0.3467100113630295
-0.4153412729501724
-0.48665428161621094
0.041431158781051636
-0.15273815393447876
-0.44216856360435486
-0.06663568317890167
None found
None found
0.10141560435295105
-0.264916256070137
-0.11757150292396545
0.24594227969646454
0.22465097904205322
-0.21580135822296143
0.0850423276424408
None found
-0.468860000371933
0.26080597937107086
-0.27672451734542847
-0.016633622348308563
None found
-0.4733085334300995
-0.4555142819881439
-0.16885997354984283
-0.47245393693447113
-0.45932963490486145
-0.4363837242126465
-0.28777456283569336
-0.22691497951745987
-0.2701912075281143
-0.2043721154332161
-0.2612279802560806
0.03470355272293091
-0.332375206053257
-0.312272846698761
0.17135386168956757
-0.31051450967788696
-0.03080582618713379
0.08220577239990234
-0.189

-0.282857209444046
-0.28062283992767334
0.022830814123153687
0.051535606384277344
-0.20788881927728653
-0.003932744264602661
-0.019735127687454224
0.16441139578819275
0.018417716026306152
None found
-0.17360132932662964
0.065492182970047
-0.24661704897880554
-0.2693120837211609
-0.1289619579911232
0.012149125337600708
-0.02285832166671753
None found
-0.31067121028900146
-0.29928727447986603
0.07998141646385193
0.08269912004470825
-0.25084954500198364
None found
None found
-0.40563148260116577
0.204665407538414
-0.13163816928863525
-0.03708767890930176
-0.3824877217411995
0.19485804438591003
-0.03816699981689453
-0.12460482120513916
-0.23962810635566711
-0.2887774407863617
-0.3052394911646843
-0.4243742525577545
-0.5533829033374786
-0.4475529342889786
-0.0937088131904602
-0.24672885984182358
-0.2886537164449692
-0.33589188009500504
-0.012378118932247162
0.2488737255334854
0.05175238847732544
0.001715824007987976
0.15149351954460144
0.07517631351947784
-0.36678117513656616
0.160884439945

0.16492806375026703
-0.05087560415267944
-0.09119647741317749
-0.28238117694854736
None found
-0.2492097020149231
0.2683144807815552
-0.3474395275115967
0.00030387938022613525
-0.0639423131942749
-0.0788881778717041
-0.13113999366760254
None found
0.2733086347579956
-0.04893433302640915
-0.052513182163238525
0.15525728464126587
-0.07185482978820801
-0.26503467559814453
-0.08867737650871277
-0.013948269188404083
-0.0498756468296051
-0.018344059586524963
-0.10350483655929565
-0.0788881778717041
-0.01406659185886383
-0.35271453857421875
None found
0.23690128326416016
0.022858381271362305
-0.12548401951789856
-0.10614228248596191
-0.06482148170471191
0.10350480675697327
-0.3421645015478134
0.3613395690917969
-0.040441691875457764
-0.4612574428319931
-0.11493398249149323
-0.3544728755950928
0.0850423276424408
-0.12020900845527649
0.2884969711303711
0.25096797943115234
-0.09110283851623535
-0.35974790155887604
-0.12669140100479126
-0.010549992322921753
0.21580135822296143
0.07596856355667114

0.08064647018909454
-0.3421645164489746
-0.25975966453552246
-0.12636318802833557
-0.2665562033653259
0.061556607484817505
-0.06406071782112122
0.04911491274833679
0.16656804084777832
-0.0702148824930191
0.17547805607318878
0.1684447079896927
-0.09647148847579956
0.2612811326980591
0.2070096731185913
0.10877993702888489
-0.1499822586774826
-0.027135729789733887
-0.1133456826210022
0.29370880126953125
0.7544255256652832
-0.10174649953842163
0.03780412673950195
-0.15601804852485657
0.17117729783058167
-0.33864784240722656
-0.24393469095230103
-0.06792104244232178
0.31403112411499023
-0.05568043142557144
-0.18239301443099976
-0.171843022108078
0.004395812749862671
-0.17108218371868134
-0.3491978645324707
-0.2562429904937744
0.13679474592208862
-0.3421645164489746
0.3280978202819824
-0.12987980246543884
0.08064651489257812
-0.07812739908695221
-0.2647978663444519
-0.3404061794281006
0.010550051927566528
-0.09998816251754761
-0.12284645438194275
0.30444860458374023
0.06306326389312744
-0.16

# Split data into training and test data

In [21]:
import splitfolders
cropped_folder = base_folder + "cropped/"

splitfolders.ratio(cropped_folder, output=base_folder + "split/", ratio=(0.8, 0.1, 0.1)) 
print("Finished!")

Copying files: 3146 files [00:00, 7517.07 files/s]

Finished!


# Retrain the imagenet network to add our shoes as classes
Based on the transfer learning done in the lab

In [29]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import os.path

MODEL_FILE = "shoes.hd5"

def create_model(num_hidden, num_classes):
    base_model = InceptionV3(include_top = False, weights = "imagenet")
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_hidden, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

def load_existing(model_file):
    model = load_model(model_file)

    numlayers = len(model.layers)

    for layer in model.layers[:numlayers-3]:
        layer.trainable = False

    for layer in model.layers[numlayers-3:]:
        layer.trainable = True

    return model

def train(model_file, train_path, validation_path, num_hidden=200, num_classes=5, steps=32, num_epochs=20):
    if os.path.exists(model_file):
        print("\nExisting model found at %s. Loading.\n\n" % model_file)
        model = load_existing(model_file)
    else:
        print("\nCreating new model.\n\n")
        model = create_model(num_hidden, num_classes)

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
    checkpoint = ModelCheckpoint(model_file)
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(train_path, target_size=(249,249), batch_size=32, class_mode="categorical")
    validation_generator = test_datagen.flow_from_directory(validation_path, target_size=(249,249), batch_size=32, class_mode="categorical")
    model.fit_generator(train_generator, steps_per_epoch = steps, epochs = num_epochs, callbacks = [checkpoint], validation_data=validation_generator, validation_steps=50)

    for layer in model.layers[:249]:
        layer.trainable = False

    for layer in model.layers[249:]:
        layer.trainable = True

    model.compile(optimizer=SGD(lr=0.00001, momentum=0.9), loss="categorical_crossentropy")

    model.fit_generator(train_generator, steps_per_epoch=steps, epochs = num_epochs, callbacks = [checkpoint], validation_data=validation_generator, validation_steps=50)

# Train the model
train_path = base_folder + "split/train/"
val_path = base_folder + "split/val/"
amt_classes = sum(os.path.isdir(train_path + f) for f in os.listdir(train_path))

train(MODEL_FILE, train_path=train_path, validation_path=val_path, num_classes=amt_classes, steps=110, num_epochs=10)

['tariq', 'rik', 'victor']
3


SyntaxError: 'break' outside loop (1733530226.py, line 72)